# Get requests to Flemish geolocation service

https://docs.python.org/3.1/howto/urllib2.html

In [63]:
import urllib.request
import urllib.parse
import json

### Details about api if of geopunt
* http://loc.geopunt.be/Help/Api/GET-v2-Location_q_latlon_xy_capakey_poi_c

* international, have a look here: http://www.latlong.net/

In [132]:
def latlon_from_address(address):
      
    url = 'http://loc.geopunt.be/v2/Location?'
    
    #strip spaces for more robustness
    values = {'q' : address.strip()}
    
    data = urllib.parse.urlencode(values)

    #get request is url + data
    #for POST request you need data.encode('utf8') first and use binary_data as a second arg instead of a '+'
    req = urllib.request.Request(url + data)
    
    req.add_header('Content-Type', 'text/json')
    response = urllib.request.urlopen(req)
    binary_response = response.read()
    decoded = binary_response.decode('utf8')
    
    response.close()

    jsonobj = json.loads(decoded)
    
    results = jsonobj['LocationResult']
    
    if len(results) == 0:
        return None
    
    location = results[0]['Location']

    latkeyID = 'Lat_WGS84'
    lonkeyID = 'Lon_WGS84'
    
    return (location[latkeyID], location[lonkeyID])
        


In [131]:
print(latlon_from_address('Ruiselede '))

(51.06302363494435, 3.38665410746602)
